# Lecture 5 - Integration: Romberg, Gauss Quadrature, Adaptive Quadrature 📊

**Learning Objectives**
* Become familiar with advanced integration methods: Romberg Integration, Adaptive Quadrature, and Gauss Quadrature.
* Compare the efficiency (# of evaluations) of these methods to Trapezoidal Rule.

## The Function for Integration
Recall that these methods require a continuous function (not discrete data points).

Implement these three integration methods to estimate:

$$ \int_0^{0.8} 0.2+25x-200x^2+675x^3-900x^4+400x^5 dx $$

The exact value is 1.640533. Compare against built-in `scipy.integrate` functions.

In [ ]:
# Define Function
def f(x):
    return 0.2 + 25*x - 200*x**2 + 675*x**3 - 900*x**4 + 400*x**5

# Define Limits of Integration
a = 0
b = 0.8

## 🤝 Romberg Integration

We will use our `trapezoid` function from last time.

In [ ]:
# Import Libraries
import numpy as np

# Trapezoidal Rule
def trapezoid(f,a,b,n):
    x = np.linspace(a,b,n+1) # n segments, n+1 points between a and b
    I = (b-a)/(2*n) * (f(x[0]) + 2*np.sum(f(x[1:-1])) + f(x[-1]))
    return I

In [ ]:
# Quick Demonstration of Romberg Integration

# Trapezoidal Rule (n=2, n=4)
I_1 = # Fill in here.
I_2 = # Fill in here.

# Romberg Integration (k=2)
I_R = # Fill in here.

# Print Results
print('I_1:', np.round(I_1,6))
print('I_2:', np.round(I_2,6))
print('I_R:', np.round(I_R,6))
print('Exact:', 1.640533)

Let's build a function that creates the Romberg Matrix. For the first column, we just use the Trapezoidal Rule:

$$I_{j,0}=\frac{h_j}{2}\bigg[ f(x_0) + 2\sum^{n-1}_{i=1}f(x_i) + f(x_n) \bigg]$$

We initialize our Romberg Matrix by specifying the maximum number of iterations we want (e.g., what is the maximum size of our Romberg Matrix) and assigning the top left value of this matrix $I_{0,0}$ by the Trapezoidal Rule.

To run the iterative Romberg Integration, we start by calculating the integral of the second row in the first column by the Trapezoidal Rule ($I_{0,1}$). This allows us to calculate the integral that is diagonally up and right of $I_{0,1}$, which would be $I_{1,0}$ by the following Romberg relationship (modified for Python indicies starting at [0,0]):

$$I_{j,k}=\frac{4^{k}I_{j+1,k-1}-I_{j,k-1}}{4^{k}-1}$$

Note that this equation will always be applied to the value that is diagonally up and right of all the preceeding values, which if $k$ is always less than or equal to $j$ (which is true for the upper triangular matrix we are constructing) Therefore, the indicies can be rewritten as:

$$I_{j-k,k}=\frac{4^{k}I_{j-k+1,k-1}-I_{j-k,k-1}}{4^{k}-1}$$


Once all the diagonal elements (starting from the second column) have been calculated with the equation above, we should check whether our integration is become more accurate within a tolerance ($tol$):

$$\big|I_{0,k} - I_{0,k-1} \big| < tol$$

If the above condition is satisfied, we can return the final value of the Romberg Matrix ($I_{0,k}$). Otherwise, we repeat this process by calculating the next value in the first columnn of our Romberg Matrix with the Trapezoidal Rule ($I_{j,0}$).

Note that the final implementation of Romberg Integration *technically* reuses function evaluations that overlap from preceeding iterations used to construct the Romberg Matrix. Therefore, the final number of function evaluats from the Trapezoidal Rule is $n+1$, which accounts for the function evaluations on either side of each step segment.

In [ ]:
# Function for Romberg Integration
def romberg(f, a, b, tol=1e-4, maxiter=10):
    R = np.zeros((maxiter, maxiter))  # Romberg matrix
    n = 1  # Initial number of segments
    R[0, 0] = trapezoid(f, a, b, n)

    for j in range(1, maxiter):         # Uses trapezoidal method to calculate
        n *= 2                          # the next row in first column
        R[j, 0] = trapezoid(f, a, b, n)

        for k in range(1, j + 1):
            R[j - k, k] = (4**k * R[j - k + 1, k - 1] - R[j - k, k - 1]) / (4**k - 1)
                                        # Calculates diagonal upwards, starting
                                        # with the second column and moving to
                                        # the upper-right most element.

        if abs(R[0, k] - R[0, k - 1]) < tol:    # Check to see if the improvement
            num_evals = n + 1                   # in the Romberg Integration is
            return R[0, k], num_evals           # less than the specified tolerance.

    # If convergence not reached.
    num_evals = n + 1
    return R[0, maxiter - 1], num_evals

Let's test our Romberg Function:

In [ ]:
# Print Result
I, num_evals = romberg(f, a, b, tol=1e-4)
print(f'Romberg: I = {I:0.7f}, num_evals = {num_evals}')

Compare to the built-in function `scipy`. If we want to see the full matrix, run with `show=True`.

The full documentation is [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.romberg.html#scipy.integrate.romberg).

In [ ]:
# Import Libraries
from scipy import integrate

# Print result from scipy function
result = integrate.romberg(f, a, b, tol=1e-4)
print(result)

# Print Full Matrix
integrate.romberg(f, a, b, tol=1e-4, show=True)

Note the DeprecationWarning. Romberg Integration appears to not have advantages over other advanced integration methods (i.e., `scipy.integrate.romberg`)

## 💪 Adaptive Quadrature

The Adaptive Quadrature starts by taking the computing the integral by the Trapezoidal Rule for two step sizes: $n=1$ and $n=2$. If these difference between these two integrals is within some tolerance, then we're done and we can compute the integral using Romberg Integration (where $k=2$):

$$I=\frac{4I_{n=2}-I_{n=1}}{3}$$

> **Note:** Running the above Romberg Integration requires 5 function evaluations: 2 for $I_{n=1}$ and 3 for $I_{n=2}$. However, the function evaluations for $I_{n=2}$ overlap with $I_{n=1}$. Therefore, there are only 3 function evaluations in total when the above Romberg Integration is computed.

If the difference between the integrals exceeds the specified tolerance, then we repeat this process by calculating the midpoint between $a$ and $b$, and repeating the above process described above. This would require use to build a "recursive function" or a function that calls upon the use of itself in order to execute.

In [ ]:
# Function for Adaptive Quadrature
def adaptive_quad(f,a,b,tol=1e-4):
  I1 = trapezoid(f,a,b,1)     # Integral for 1 segment: 2 evaluations
  I2 = trapezoid(f,a,b,2)     # Integral for 2 segments: 3 evaluations, but 2 are reused.

  if abs(I2-I1) < tol:
    return (4*I2 - I1)/3, 3   # Romberg Integral, # of Evaluations

  else:
    # Begin by calculating the midpoint
    # Use this midpoint to call adaptive_quad twice:
    #     Once from a to the midpoint
    #     And again from the midpoint to b
    # Call the outputs Ia, na and Ib, nb respectively.
    return Ia + Ib, na + nb

Let's test our Adaptive Quadrature Function:

In [ ]:
# Print Result
I,num_evals = adaptive_quad(f, a, b, tol=1e-4)
print(f'Adaptive: I = {I:0.7f}, num_evals = {num_evals}')

Compare to built in `quad`. Documentation [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html#scipy.integrate.quad)

In [ ]:
# Print result from scipy function
result, err, infodict = integrate.quad(f, a, b, epsabs=1e-4, full_output=True)
print(result)
print('num_evals:', infodict['neval'])

The built-in function is much more efficient and accurate than our simplified version because it uses an "Adaptive Gauss" method, as opposed to our "Trapezoidal-Romberg" method.

## 💪 Gauss Quadrature

<p align="center">
  <img src="https://github.com/cdefinnda/ECI-115_HW-Images/blob/main/HW3_Table22.1.png?raw=true" alt="Table 22.1" width=500>
</p>
</font>

The Gauss Quadrature uses weighting factors ($c_i$) and function arguments ($x_i$) that are specified in Table 22.1. Once we have these values, we then need to scale the $x_i$ values from a range of [-1,1] to $y_i$ from a range [$a$,$b$] by:
$$y_i=\frac{a+b}{2}+\frac{b-a}{2}x_i$$

Then we can compute the integral by using the following approximation:

$$\int_a^b f(y)\ dy \approx \Big(\frac{b-a}{2}\Big)\sum^N_{i=0}c_if(y_i)$$


Write a function that computes the 3-Point Gauss Quadrature:

In [ ]:
def gauss_3(f,a,b):
  # Define guass weights and points
  c = np.array([])  # Fill in code here.
  x = np.array([])  # Fill in code here.

  # Map gauss points (x) from the interval [-1,1] to y values on the interval [a,b]
  y =   # Fill in code here.

  # Compute the gauss integral
  I =   # Fill in code here
  num_evals =   # Fill in code here.
  return I, num_evals

Let's test our 3-Point Gauss Quadrature Function:

In [ ]:
# Print Result
I , num_evals = gauss_3(f,a,b)
print(f'3-Point Gauss: I = {I:0.7f}, num_evals = {num_evals}')

We can write similar functions for 2, 4, 5, 6-point Gauss quadrature using the values from Table 22.1 of the textbook. For now, use the built-in `fixed_quad`. Specify `n=` the number of points. Docs [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.fixed_quad.html#scipy.integrate.fixed_quad).

In [ ]:
# Print result from scipy function
for n in range(2,10):
    I,num_evals = integrate.fixed_quad(f, a, b, n=n)
    print(f'{n}-point Gauss Quadrature: {I:0.7f}')

Notice that 3-point Gauss should exactly integrate a $2N+1=5$th order polynomial, so in this case the answer is perfect. For a general $f(x)$ that is not a polynomial, it wouldn't work quite so well.

## 💪 Compare to Trapezoidal Rule

##### Compare accuracy to reach `tol=1e-4`
- Romberg: 9 function evaluations
- Adaptive: 21 function evaluations (with built-in function, ours was 129)
- Gauss: 3 function evaluations (special case for 5th order polynomial)

Write a `while` loop the computes the integral by the Trapezoidal Rule for step sizes increasing by $n=2^i$.

Start by initializing `I_old` and `I_new` by using the Trapezoidal Rule for $n=2^0=1$ and $n=2^1=2$ respectively. Then create a `while` loop that compares the absolute difference between these two integrals to the `tol=1e-4` (note: the `while` loop should run until this difference is less than `tol`). Be sure to update values for `I_old`, `n`, and `I_new` within the `while` loop.

In [ ]:
# Initialized tol, I_old, n, I_new
tol = 1e-4
I_old =   # Fill in code here.
n =       # Fill in code here.
I_new =   # Fill in code here.

# While loop that will run as long as abs(I_new - I_old) >= tol
# Build while loop here.

# Compute final value of I_trap with n updated by the while loop
I_trap =      # Fill in code here.
num_evals =   # Fill in code here.

# Print Final Result
print(f'Trapezoidal Rule: I = {I_trap:0.7f}, num_evals = {num_evals}')